mypy에서 이전에 만든 CohesionProbability를 가져오겠습니다. 

sys.path에 '../mypy/'를 추가합니다. 

In [1]:
import sys
sys.path.append('../mypy/')

In [2]:
from cohesion import CohesionProbability

cohesion = CohesionProbability(min_count=5)

data 폴더에서 company_type_list.txt를 로딩합니다. 각 기업에 대하여 수기로 입력된 업태 데이터입니다. 

형태는 "업태\t업종"입니다

In [3]:
def load_business_type_field(fname):
    with open(fname, encoding='utf-8') as f:
        business_type, business_field = zip(*(doc.replace('\n', '').split('\t') for doc in f))
    return business_type, business_field
    
business_type, business_field = load_business_type_field('../../../data/company_name/company_type_list.txt')
    
print(business_type[:5])
print(business_field[:5])

('도소매,서비스', '제조업,건설업', '도소매,서비스', '도소매,서비스', '서비스')
('전자상거래업, 소프트웨어개발', '컴퓨터주변기기외', '컴퓨터및주변기기', '컴퓨터및주변기기', '소프트웨어개발공급및유지보수')


github에서 clone을 해둔 package를 이용하기 위하여 sys.path에 폴더를 하나 더 추가합니다. 

In [4]:
sys.path.append('../soy/')
import soy

normalize라는 함수를 import 해왔습니다. 

In [11]:
from soy.nlp.hangle import normalize

business_type = [normalize(doc) for doc in business_type]
business_field = [normalize(doc) for doc in business_field]
print(business_type[:5])
print(business_field[:5])

['도소매 서비스', '제조업 건설업', '도소매 서비스', '도소매 서비스', '서비스']
['전자상거래업 소프트웨어개발', '컴퓨터주변기기외', '컴퓨터및주변기기', '컴퓨터및주변기기', '소프트웨어개발공급및유지보수']


normalize는 한글/영어/숫자/특수기호 등을 필요에 따라서 남겨두는 전처리 함수입니다. 지워지는 글자는 빈칸으로 처리합니다. 두 개 이상 연속된 빈칸은 모두 지워둡니다. 

In [8]:
s = '이건123숫자abc영어가모두섞인 문장!! '
normalize(s, number=True)

'이건123숫자 영어가모두섞인 문장'

default value는 아래와 같습니다. 

    normalize(doc, english=False, number=False, punctuation=False, remove_repeat=0, remains={})

In [9]:
normalize(s)

'이건 숫자 영어가모두섞인 문장'

In [10]:
normalize(s, english=True)

'이건 숫자abc영어가모두섞인 문장'

먼저 제대로 입력된 업태들을 살펴보기 위하여 Counter를 이용하여 빈도수가 높은 어절들을 살펴보겠습니다. 

서비스, 제조, 세관, 도소매 등 제대로 된 단어들도 많이 있지만, 서비스외, 제조외와 같이 -외로 끝나는 단어들이 있습니다. 또한 오탈자인 써비스도 존재합니다. 

좀 더 아래로 내려가면 운수관련서비스와 같이 다른 업태의 합성어인 업태들이 있습니다. 

In [12]:
from collections import Counter

business_type_tokens = Counter([token for doc in business_type for token in doc.split()])

print('num token in business type = %d' % len(business_type_tokens))
for token, freq in sorted(business_type_tokens.items(), key=lambda x:x[1], reverse=True)[:50]:
    print('%10s\t\t%d' % (token, freq))


num token in business type = 158
       서비스		7186
        제조		4298
        세관		3025
       도소매		2197
      서비스외		1155
        도매		1112
        소매		818
       부동산		815
         외		791
       제조업		767
         도		429
        통신		362
    사업서비스외		352
       제조외		309
        건설		297
      운수관련		249
        운보		223
      서비스업		208
      도소매외		196
       건설업		157
       통신업		144
        운수		119
       도매업		116
       써비스		113
       소매외		101
       운수업		100
      부동산업		83
      통신업외		83
       소매업		61
     전기통신업		61
        보관		55
      전기가스		51
   운수관련서비스		45
       운보외		44
         및		42
       음숙외		40
     다단계판매		39
       도매외		38
   부동산업부동산		37
        음숙		36
      기타관련		35
     상품중개업		34
       건설외		33
      도소매업		30
      부동산외		27
        농업		26
      사업관련		26
         업		26
       비영리		25
      제조업외		25


CohesionProbability를 학습시켜 단어를 추출하겠습니다. 

In [13]:
cohesion.train(business_type)

inserting subwords into L: done
num subword = 360
num subword = 229 (after pruning with min count 5)


In [14]:
print('%12s\t\t(freq, cohesion_l, cohesion_r)\n%s' % ('단어', '-'*50))
for token, freq in sorted(business_type_tokens.items(), key=lambda x:x[1], reverse=True):
    cohesion_l = cohesion.get_cohesion(token)
    freq = cohesion.L.get(token, 0)
    
    if cohesion_l < 0.1 or freq < 10 or len(token) == 1:
        continue
    print('%12s\t\t(%4d, %.3f)' % (token, freq, cohesion_l))


          단어		(freq, cohesion_l, cohesion_r)
--------------------------------------------------
         서비스		(8599, 0.998)
          제조		(5436, 0.998)
          세관		(3025, 1.000)
         도소매		(2441, 0.766)
        서비스외		(1155, 0.512)
          도매		(1277, 0.307)
          소매		( 980, 0.998)
         부동산		( 971, 1.000)
         제조업		( 808, 0.385)
          통신		( 604, 1.000)
      사업서비스외		( 352, 0.955)
         제조외		( 309, 0.238)
          건설		( 494, 0.998)
        운수관련		( 294, 0.710)
          운보		( 274, 0.334)
        서비스업		( 225, 0.297)
        도소매외		( 196, 0.361)
         건설업		( 161, 0.570)
         통신업		( 227, 0.613)
          운수		( 527, 0.643)
         도매업		( 117, 0.168)
         써비스		( 113, 1.000)
         소매외		( 101, 0.321)
         운수업		( 101, 0.351)
        부동산업		( 120, 0.498)
        통신업외		(  83, 0.516)
         소매업		(  61, 0.249)
       전기통신업		(  61, 0.859)
          보관		(  56, 0.903)
        전기가스		(  51, 0.769)
     운수관련서비스		(  45, 0.616)
         운보외		(  44, 0.232)
        

초/중/종성을 분해하는 jamo_levenshtein을 이용하기 위하여 string_distance에서 import 해옵니다. 

string_distance.py는 ../mypy/에 들어있습니다. 

In [15]:
from string_distance import jamo_levenshtein

jamo_levenshtein('아니야', '아니얌')

0.3333333333333333

match 함수는 주어진 token에 대하여 현재까지 단어로 알려진 dictionary를 이용하여, 길이가 2보다 긴 단어들의 조합으로 이뤄진 token을 단어로 분해한 것입니다. 

예를 들어 
    
    {
        '도소매': 100,
        '서비스': 50, 
        '소매': 30
    }
    
이 알려진 dictionary일 경우, '도소매서비스판매'에 대하여, 빈도수가 가장 높은 단어 '도소매'가 들어있기 때문에 token을 아래와 같이 변형시킵니다. 

    '도소매서비스판매' --> '[도소매]서비스판매'
    
[, ]로 경계를 친 이유는, 이후에 '소매'라는 단어가 

    '[도소매]서비스판매' --> '[도[소매]]서비스판매'
    
처럼 만드는 일을 막기 위함입니다. 

사전의 모든 단어에 대하여 확인이 끝나면 [, ]부분에 split()을 하여 token을 단어로 나눕니다. 이를 통하여 여러 업태의 합성어로 이뤄진 업태를 토크나이징합니다. 

In [16]:
def match(token, dictionary):
    for word, _ in sorted(dictionary.items(), key=lambda x:x[1], reverse=True):
        if (word in token) and ( ('[%s'%word in token) == False ) and ( ('%s]'%word in token) == False):
            token = token.replace(word, '[%s]' % word)
            
    # 제조 서비스 업 -> 제조 서비스
    token = token.replace('[', ']').replace(']]',']')
    token = ' '.join([t for t in token.split(']') if len(t) >= 2])
    return token.strip()

for token in ['도소매서비스판매', '금융서비스']:
    print('%s --> %s' % (token, match(token, {'도소매':100, '서비스':50, '소매':30})))

도소매서비스판매 --> 도소매 서비스 판매
금융서비스 --> 금융 서비스


find_similars 함수는 dictionary에 들어있는 단어들 중에서 token과 jamo_levenshtein 거리가 <= 0.5 이하인 단어를 찾는 함수입니다. 거리값은 1/3 단위로 변하기 때문에 거리 <=0.5는 초/중/종성중 하나 이상 다르지 않다는 의미입니다. 

dictionary 안에 두 개 이상으로 유사한 단어가 있다면 빈도수가 가장 높은 단어를 선택합니다. 만약 비슷한 단어가 없으면 None을 return 합니다. 

In [17]:
def find_similars(token, dictionary):
    if not dictionary:
        return None
    
    similars = {word:jamo_levenshtein(token, word) for word in dictionary.keys()}
    min_distance = min(similars.values())
    if min_distance > 0.5:
        return None
    
    similars = {word:dist for word, dist in similars.items() if dist == min_distance}
    return sorted(similars.keys(), key=lambda x:dictionary.get(x, 0), reverse=True)[0]

print(find_similars('써비스', {'서비스':100}))
print(find_similars('자비스', {'서비스':100}))

서비스
None


업태의 표현들을 제대로 된 단어들과 노이즈로 나누는 작업을 수행할 겁니다. 

business_type_dictionary = {}는 빈 dict로 시작하여, 제대로 된 단어들을 체울 것이며, business_type_noisy = {}는 빈 dict로 시작하여, 노이즈들을 추가할 겁니다. 

business_type_dictionary는 {단어:빈도수} 형태로 입력할겁니다. business_type_noisy은 {오탈자:정자} 형태로 입력할 겁니다. {'써비스': '서비스'} 입니다. 

passwords는 이미 단어로 알려진 업태입니다. passwords = ['소매', '도매', '도소매', '금융']로 네 개의 단어만 알고 있다고 가정합니다. 

    for password in passwords:
        business_type_dictionary[password] = business_type_tokens[password]
        
correct_noise 함수는 tokens, dictionary, noisy, 세 가지 dict를 받아서, dictionary, noisy를 return 합니다. 

correct_noise 함수는 token을 여섯가지 경우로 나눠서 생각합니다. 

1. 첫번째는 길이가 1보다 작은 단어로, 이는 제거합니다. 
1. 두번째는 "서비스외 --> 서비스"로 고치는 경우로, 뒤의 한 글자를 제외하면 이미 dictionary에 등록이 되어있는 단어입니다. 
1. 세번째는 "써비스업외 --> 서비스 + 업외"처럼 나뉘어지는 경우로, 뒤의 두글자를 제외하면 단어가 dictionary에 등록이 되어있지만, 뒤의 두글자는 등록이 되어있지 않는 경우입니다. 
1. 네번째는 "도매서비스 --> 도매 + 서비스"처럼 두 개 이상의 업태 단어가 합성된 경우입니다. 앞의 match 함수를 이용하여 띄어쓰기가 추가가 되면 이는 합성어로 판단하여 수정합니다. 
1. 다섯번째는 "써비스 -> 서비스"처럼 dictionary에 jamo_levenshtein 거리가 0.5 이하인 단어가 존재하는 경우입니다. 이 때에는 dictionary에 존재하는 단어로 치환합니다. 
1. 그 외의 단어는 dictionary에 등록하여 다음 단어로 넘어갑니다. 

correct_noise 함수는 주어진 tokens에 대하여 빈도수 기준으로 내림정렬을 한 뒤, 빈도수가 높은 단어부터 위 경우 중 어떤 경우에 해당하는지 살펴봅니다. 

즉, 이 함수는 빈도수가 높은 단어부터 유사하거나 합성어가 아닌 경우, 올바른 단어로 추가하는 것입니다. 

In [18]:
business_type_dictionary = {} # word: freq
business_type_noisy = {} # noisy word to corrected word

# exception
passwords = ['소매', '도매', '도소매', '금융']
for password in passwords:
    business_type_dictionary[password] = business_type_tokens[password]

def correct_noise(tokens, dictionary, noisy):
    for num, (token, freq) in enumerate(sorted(tokens.items(), key=lambda x:x[1], reverse=True)):
        if (num + 1) % 100 == 0:
            print('%d in %d' % (num + 1, len(tokens)))
        
        if len(token) <= 1:
            business_type_noisy[token] = ''
            continue

        # check 서비스외 = 서비스 + 외
        if (token[:-1] in dictionary):
            noisy[token] = token[:-1]
            continue

        # check 서비스업외 = 서비스 + 업외
        if (len(token) > 2) and (token[:-2] in dictionary) and ((token[-2:] in dictionary) == False):
            noisy[token] = token[:-2]
            continue

        # check 도매서비스 = 도매 + 서비스
        matched_token = match(token, dictionary)
        if ' ' in matched_token:
            noisy[token] = matched_token
            continue

        # check 써비스 -> 서비스
        most_similar = find_similars(token, dictionary)
        if (most_similar != None) and (most_similar != token):
            noisy[token] = most_similar
            continue

        # Add token into dictionary
        dictionary[token] = freq
        
    return dictionary, noisy


business_type_dictionary, business_type_noisy = correct_noise(business_type_tokens, business_type_dictionary, business_type_noisy)
print('all tokens are parsed?', len(business_type_dictionary) + len(business_type_noisy) == len(business_type_tokens))

100 in 158
all tokens are parsed? True


business_type_dictionary에 등록된 단어입니다. '음숙외'와 같은 단어가 사전에 들어있습니다. 이는 '음숙'이 36번으로 '음숙외'보다 덜 등장했기 때문입니다. 

In [19]:
sorted(business_type_dictionary.items(), key=lambda x:x[1], reverse=True)

[('서비스', 7186),
 ('제조', 4298),
 ('세관', 3025),
 ('도소매', 2197),
 ('도매', 1112),
 ('소매', 818),
 ('부동산', 815),
 ('통신', 362),
 ('건설', 297),
 ('운수관련', 249),
 ('운보', 223),
 ('운수', 119),
 ('보관', 55),
 ('전기가스', 51),
 ('음숙외', 40),
 ('다단계판매', 39),
 ('음숙', 36),
 ('기타관련', 35),
 ('상품중개업', 34),
 ('농업', 26),
 ('사업관련', 26),
 ('비영리', 25),
 ('자유직업', 21),
 ('서비', 21),
 ('사회', 20),
 ('운송', 19),
 ('해운외', 19),
 ('도난방지기및감시장치', 17),
 ('연구개발컨설팅', 16),
 ('유리제품', 14),
 ('유리및', 14),
 ('정보동신', 13),
 ('교육', 13),
 ('작물재배', 9),
 ('의료학회', 8),
 ('사단법인', 8),
 ('출판', 8),
 ('음식업', 7),
 ('숙박업', 7),
 ('보건업', 5),
 ('학교', 5),
 ('자동자제외', 4),
 ('의료업', 2),
 ('데이터베이스', 2),
 ('판매', 2),
 ('브랜드개발', 2),
 ('광고기획', 2),
 ('정화조청소', 2),
 ('소도매', 1),
 ('의료', 1),
 ('숙박', 1),
 ('해운업', 1),
 ('금융', 1),
 ('비영', 1),
 ('화물취급업', 1),
 ('건서업', 1),
 ('무선전화업', 1),
 ('제작', 1)]

business_type_noisy에 들어있는 단어들이 어떻게 오탈자가 수정되었는지 살펴보겠습니다. 

    건설제조업건설업서비스 (1)	-->	건설 제조 건설 서비스
    
처럼 여러 개의 합성어로 이뤄진 업태도 분해가 됩니다. 


In [20]:
for noisy_token, corrected in sorted(business_type_noisy.items(), key=lambda x:business_type_tokens.get(x[0], 0), reverse=True):
    print('%10s (%d)\t-->\t%s' % (noisy_token, business_type_tokens[noisy_token], corrected))

      서비스외 (1155)	-->	서비스
         외 (791)	-->	
       제조업 (767)	-->	제조
         도 (429)	-->	
    사업서비스외 (352)	-->	사업 서비스
       제조외 (309)	-->	제조
      서비스업 (208)	-->	서비스
      도소매외 (196)	-->	도소매
       건설업 (157)	-->	건설
       통신업 (144)	-->	통신
       도매업 (116)	-->	도매
       써비스 (113)	-->	서비스
       소매외 (101)	-->	소매
       운수업 (100)	-->	운수
      부동산업 (83)	-->	부동산
      통신업외 (83)	-->	통신
       소매업 (61)	-->	소매
     전기통신업 (61)	-->	전기 통신
   운수관련서비스 (45)	-->	운수관련 서비스
       운보외 (44)	-->	운보
         및 (42)	-->	
       도매외 (38)	-->	도매
   부동산업부동산 (37)	-->	부동산 부동산
       건설외 (33)	-->	건설
      도소매업 (30)	-->	도소매
      부동산외 (27)	-->	부동산
         업 (26)	-->	
      제조업외 (25)	-->	제조
     법무서비스 (24)	-->	법무 서비스
    서비스도소매 (23)	-->	서비스 도소매
   자동차제조업등 (20)	-->	자동차 제조 업등
   식품제조가공업 (19)	-->	식품 제조 가공업
    도소매서비스 (17)	-->	도소매 서비스
     서비스업외 (17)	-->	서비스
       금융업 (16)	-->	금융
    사업서비스업 (16)	-->	사업 서비스
        음식 (15)	-->	음숙
    금융및보험업 (14)	-->	금융 및보험업
       통신외 (14)	-->	통신
     사업서비스 (13)	-->	사업 서비스
      

동일한 작업을 업종에 대해서도 수행해 보겠습니다. 

In [18]:
business_field_tokens = Counter([token for doc in business_field for token in doc.split()])

business_field_dictionary = {} # word: freq
business_field_noisy = {} # noisy word to corrected word

# exception
passwords = ['소매', '도매', '도소매', '금융']
for password in passwords:
    business_field_dictionary[password] = business_field_tokens[password]

business_field_dictionary, business_field_noisy = correct_noise(business_field_tokens, business_field_dictionary, business_field_noisy)

100 in 1238
200 in 1238
300 in 1238
400 in 1238
500 in 1238
600 in 1238
700 in 1238
800 in 1238
900 in 1238
1000 in 1238
1100 in 1238
1200 in 1238


In [19]:
sorted(business_field_dictionary.items(), key=lambda x:x[1], reverse=True)

[('세관', 3025),
 ('지류외', 959),
 ('화물운송대행', 801),
 ('배송센터외', 667),
 ('임대', 636),
 ('관세사', 551),
 ('인쇄', 495),
 ('커피', 460),
 ('컴퓨터및주변기기', 397),
 ('무역', 382),
 ('다단계판매', 359),
 ('전신전화', 341),
 ('라면', 326),
 ('음료외', 326),
 ('출판', 305),
 ('광고대행', 292),
 ('창고보관', 249),
 ('생화', 243),
 ('광고', 233),
 ('발효유제품', 213),
 ('하역', 200),
 ('다단계', 200),
 ('소프트웨어개발', 194),
 ('건강보조식품', 158),
 ('액상시유', 158),
 ('복합운송주선업', 158),
 ('연유', 158),
 ('운송알선', 156),
 ('차량대여', 156),
 ('타이어', 156),
 ('후원수당', 137),
 ('프로그램개발공급업', 130),
 ('인터넷정보제공', 123),
 ('이벤트', 117),
 ('전시', 114),
 ('신변잡화', 110),
 ('의약품', 105),
 ('인터넷판매', 101),
 ('디자인', 96),
 ('컴퓨터주변기기외', 95),
 ('부동산매매', 94),
 ('영상회의시스템외', 89),
 ('통신장비', 87),
 ('플라스틱', 83),
 ('기획', 82),
 ('개인휴대통신외', 82),
 ('화장품', 82),
 ('공장재단', 80),
 ('대여', 80),
 ('광업재단', 80),
 ('전산용지외', 79),
 ('컨설팅', 74),
 ('번역', 74),
 ('개발', 74),
 ('사무기기', 73),
 ('재판', 72),
 ('사진촬영', 72),
 ('수세미', 72),
 ('서비스', 71),
 ('홍보대행', 70),
 ('통역', 70),
 ('패키징', 70),
 ('온라인정보제공', 69),
 ('근로자파견업외', 68),
 ('호텔

In [20]:
for noisy_token, corrected in sorted(business_field_noisy.items(), key=lambda x:business_field_tokens.get(x[0], 0), reverse=True):
    print('%16s (%d)\t-->\t%s' % (noisy_token, business_field_tokens[noisy_token], corrected))

          다단계판매원 (188)	-->	다단계판매
            화분임대 (184)	-->	화분 임대
            광고선전 (114)	-->	광고
      인쇄출판디자인기획외 (91)	-->	인쇄 출판 디자인 기획외
  산업단지의개발조성및분양임대 (80)	-->	산업단지의개발조성및분양 임대
           광고대행외 (75)	-->	광고대행
       소프트웨어개발공급 (69)	-->	소프트웨어개발
          개발및판매외 (67)	-->	개발 및판매외
          다단계후원수 (65)	-->	다단계 후원수
      소프트웨어개발및공급 (64)	-->	소프트웨어개발 및공급
            컨설팅외 (62)	-->	컨설팅
            수세미외 (60)	-->	수세미
           의약품제조 (55)	-->	의약품
    소프트웨어프로그램개발외 (54)	-->	소프트웨어 프로그램 개발
           디지털인쇄 (53)	-->	디지털 인쇄
        소프트웨어개발외 (52)	-->	소프트웨어개발
          종합유선방송 (48)	-->	종합 유선 방송
           옵셋인쇄외 (46)	-->	옵셋 인쇄
         소프트웨어자문 (43)	-->	소프트웨어
         다단계후원수당 (40)	-->	다단계 후원수당
              전기 (37)	-->	전시
          유선통신장비 (37)	-->	유선 통신장비
          위탁대리판매 (33)	-->	위탁대리 판매
          광고물제작외 (33)	-->	광고 제작
            유선방송 (32)	-->	유선
             인쇄외 (31)	-->	인쇄
         기술검사서비스 (29)	-->	기술검사 서비스
           주차장운영 (28)	-->	주차장
            개발용역 (28)	-->	개발
          원부자재무역 (28)	-->	원부자재 무역
     